In [1]:
import urllib2
import json
import datetime
import csv
import time

In [2]:
# SCRAPPER APP
app_id = "351254375231342"
app_secret = "cf4916b91710cd1a5c749066e216af08" 

access_token = app_id + "|" + app_secret

In [3]:
# open pages.txt file containing all relevant companys + facebook id's
pages = {}
with open('pages.txt', 'r') as f:
    for line in f:
        splitLine = line.split('|')
        pages[splitLine[0]] = splitLine[1].replace('\n', '')

In [4]:
# manual page_ids

#page_id = 'arend40'
#page_id = '1380503632189127'

In [5]:
# only for connection/response testing purposes
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + '/feed'
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)

for page_id in pages.iterkeys():
    pass
    #testFacebookPageData(str(page_id), access_token)
    

In [6]:
# if exception, wait 5 seconds and try again
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [7]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

#test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
#print json.dumps(test_status, indent=4, sort_keys=True)

In [8]:
def processFacebookPageFeedStatus(status, company):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (company, status_id, status_message, link_name, status_type, status_link,
           status_published, likes, comments, shares)

#processed_test_status = processFacebookPageFeedStatus(test_status)
#print processed_test_status

In [9]:
def scrapeFacebookPageFeedStatus(page_id, access_token, company):
    with open('%s_facebook.csv' % company, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["company", "status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "likes", "comments", "shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (company, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status, company))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)

for page_id, name in pages.iteritems():
    scrapeFacebookPageFeedStatus(page_id, access_token, name)

Scraping beckhoff Facebook Page: 2017-01-10 11:09:55.984391


Done!
477 Statuses Processed in 0:00:06.245782
Scraping köhl Facebook Page: 2017-01-10 11:10:02.238844


Done!
69 Statuses Processed in 0:00:00.960886
Scraping arend Facebook Page: 2017-01-10 11:10:03.207538


Done!
80 Statuses Processed in 0:00:00.993231
Scraping blumenbecker Facebook Page: 2017-01-10 11:10:04.208499


Done!
579 Statuses Processed in 0:00:06.602422
Scraping natus Facebook Page: 2017-01-10 11:10:10.819466


Done!
9 Statuses Processed in 0:00:00.577010
Scraping pilz Facebook Page: 2017-01-10 11:10:11.404408



UnicodeEncodeError: 'ascii' codec can't encode characters in position 74-137: ordinal not in range(128)